In [1]:
import http.client
import json
import pandas as pd


connection = http.client.HTTPConnection('api.football-data.org')
headers = { 'X-Auth-Token': 'b55a7652a9e9403dbbd0d0da87c17c14', 'X-Response-Control': 'minified' }
connection.request('GET', '/v1/competitions/467/teams', None, headers )
response = json.loads(connection.getresponse().read().decode())

In [2]:
print(json.dumps(response, indent=4, sort_keys=True))

{
    "count": 32,
    "teams": [
        {
            "crestUrl": "https://upload.wikimedia.org/wikipedia/commons/f/f3/Flag_of_Russia.svg",
            "id": 808,
            "name": "Russia",
            "shortName": null,
            "squadMarketValue": null
        },
        {
            "crestUrl": "https://upload.wikimedia.org/wikipedia/commons/0/0d/Flag_of_Saudi_Arabia.svg",
            "id": 801,
            "name": "Saudi Arabia",
            "shortName": null,
            "squadMarketValue": null
        },
        {
            "crestUrl": "https://upload.wikimedia.org/wikipedia/commons/f/fe/Flag_of_Egypt.svg",
            "id": 825,
            "name": "Egypt",
            "shortName": null,
            "squadMarketValue": null
        },
        {
            "crestUrl": "https://upload.wikimedia.org/wikipedia/commons/f/fe/Flag_of_Uruguay.svg",
            "id": 758,
            "name": "Uruguay",
            "shortName": null,
            "squadMarketValue": null
     

In [3]:
name = []
# team_id = []
for x in range(0,32):
    
    name.append(response["teams"][x]["name"])
#     team_id.append(response["teams"][x]["id"])
    
team_dict = {"Country": name,}
team_data_df = pd.DataFrame(team_dict)
team_data_df.head(32)

,Country
0,Russia
1,Saudi Arabia
2,Egypt
3,Uruguay
4,Morocco
5,Iran
6,Portugal
7,Spain
8,France
9,Australia


In [4]:
types_df = pd.read_csv("./results.csv")
types_df.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [6]:
types_df.columns
types_df["away_score"].sum()

46065